<a href="https://colab.research.google.com/github/aserravalle/Fraud-Detection-Logistic-Regression/blob/master/Fraud_Detection_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Kaggle

In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle, so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle competitions download -c ieee-fraud-detection

 78% 41.0M/52.5M [00:00<00:00, 50.4MB/s]
100% 52.5M/52.5M [00:00<00:00, 82.6MB/s]
  0% 0.00/3.02M [00:00<?, ?B/s]
100% 3.02M/3.02M [00:00<00:00, 98.1MB/s]
 93% 44.0M/47.3M [00:00<00:00, 45.3MB/s]
100% 47.3M/47.3M [00:00<00:00, 69.3MB/s]
  0% 0.00/2.97M [00:00<?, ?B/s]
100% 2.97M/2.97M [00:00<00:00, 197MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 160MB/s]


## Dependencies

In [4]:
! pip install kmeans_smote 

     |████████████████████████████████| 5.4MB 6.9MB/s 
     |████████████████████████████████| 13.9MB 37.0MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.5
    Uninstalling numpy-1.16.5:
      Successfully uninstalled numpy-1.16.5
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3


In [0]:
import pandas as pd
import numpy as np
from datetime import datetime
from math import ceil

# Plotting
import matplotlib.pyplot as plt
plt.style.use('dark_background')

# Dimensionality reduction and SMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from kmeans_smote import KMeansSMOTE

# Model Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

# Modelling
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

# Options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

# Target variable = 'isFraud'

## Reading Files

In [6]:
train_id = pd.read_csv('/content/train_identity.csv.zip', nrows=20000)
print(train_id.shape)
test_id = pd.read_csv('/content/test_identity.csv.zip', nrows=20000)
print(test_id.shape)
train_id.head()

(20000, 41)
(20000, 41)


,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [7]:
train_trans = pd.read_csv('/content/train_transaction.csv.zip', nrows=20000)
print(train_trans.shape)
test_trans = pd.read_csv('/content/test_transaction.csv.zip', nrows=20000)
print(test_trans.shape)

train_trans.head()

(20000, 394)
(20000, 393)


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,

# Cleaning

## Define fctn

In [0]:
def Clean(df):
    # Drop columns w many missing values
    thresh = ceil(len(df) * 0.4)
    cols = df.columns[df.isnull().sum() > thresh]
    df.drop(cols, axis = 1, inplace = True)
    print('DROP: ', [f+',' for f in cols])

    # Determine types of features
    cat_features = [f for f in df.columns if (np.dtype(df[f]) == 'object') & (f != 'TransactionID')]
    num_features = [f for f in df.columns if (np.dtype(df[f]) != 'object') & (f != 'TransactionID')]

    # Impute missing
    df2 = pd.DataFrame()
    df2[num_features] = df[num_features].fillna(df[num_features].mean())
    print('IMPUTE: ', [f+',' for f in cols])

    # OHE categorical
    for f in cat_features:
        dummies = pd.get_dummies(df[f], prefix = f, prefix_sep = ': ', dummy_na = True)
        df2 = pd.concat([df2, dummies], axis = 1)
    print('OHE: ', [f+',' for f in cat_features])

    # Get the ID back
    df2['TransactionID'] = df['TransactionID']
    print('Cleaned\n')
    return df2

In [9]:
train_id_clean = Clean(train_id)
test_id_clean = Clean(test_id)
train_trans_clean = Clean(train_trans)
test_trans_clean = Clean(test_trans)

DROP:  ['id_03,', 'id_04,', 'id_07,', 'id_08,', 'id_09,', 'id_10,', 'id_18,', 'id_21,', 'id_22,', 'id_23,', 'id_24,', 'id_25,', 'id_26,', 'id_27,']
IMPUTE:  ['id_03,', 'id_04,', 'id_07,', 'id_08,', 'id_09,', 'id_10,', 'id_18,', 'id_21,', 'id_22,', 'id_23,', 'id_24,', 'id_25,', 'id_26,', 'id_27,']
OHE:  ['id_12,', 'id_15,', 'id_16,', 'id_28,', 'id_29,', 'id_30,', 'id_31,', 'id_33,', 'id_34,', 'id_35,', 'id_36,', 'id_37,', 'id_38,', 'DeviceType,', 'DeviceInfo,']
Cleaned

DROP:  ['id_03,', 'id_04,', 'id_07,', 'id_08,', 'id_09,', 'id_10,', 'id_14,', 'id_18,', 'id_21,', 'id_22,', 'id_23,', 'id_24,', 'id_25,', 'id_26,', 'id_27,', 'id_30,', 'id_32,', 'id_33,', 'id_34,']
IMPUTE:  ['id_03,', 'id_04,', 'id_07,', 'id_08,', 'id_09,', 'id_10,', 'id_14,', 'id_18,', 'id_21,', 'id_22,', 'id_23,', 'id_24,', 'id_25,', 'id_26,', 'id_27,', 'id_30,', 'id_32,', 'id_33,', 'id_34,']
OHE:  ['id_12,', 'id_15,', 'id_16,', 'id_28,', 'id_29,', 'id_31,', 'id_35,', 'id_36,', 'id_37,', 'id_38,', 'DeviceType,', 'Devic

## Remove Cols
that are not in both training and testing data

In [10]:
# Get a set of the columns in each dataframe
a = set(train_id_clean.columns)
print('train',len(a))
b = set(test_id_clean.columns)
print('test',len(b))

# Take the intersection
cols = list(a.intersection(b))
print('columns',len(cols))

# Set columns for both data frames equal to the set 
train_id_eql = train_id_clean.loc[:,cols]
test_id_eql = test_id_clean.loc[:,cols]

print('new train ',len(train_id_eql.columns))
print('new test ',len(test_id_eql.columns))

train 926
test 966
columns 401
new train  401
new test  401


In [11]:
# Get a set of the columns in each dataframe
a = set(train_trans_clean.columns)
print('train',len(a))
b = set(test_trans_clean.columns)
print('test',len(b))

# Take the intersection
cols = list(a.intersection(b))
print('columns',len(cols))

# Set columns for both data frames equal to the set 
train_trans_eql = train_trans_clean.loc[:,cols] 
train_trans_eql['isFraud'] = train_trans_clean['isFraud'] # Remember to keep the target variable
test_trans_eql = test_trans_clean.loc[:,cols]

print('new train ',len(train_trans_eql.columns))
print('new test ',len(test_trans_eql.columns), '(missing isFraud)')

train 233
test 302
columns 229
new train  230
new test  229 (missing isFraud)


In [12]:
print(train_id_eql.shape)
print(test_id_eql.shape)
print(train_trans_eql.shape)
print(test_trans_eql.shape)

(20000, 401)
(20000, 401)
(20000, 230)
(20000, 229)


## Merge datasets

In [13]:
print(train_id_eql.shape, train_trans_eql.shape)

train_clean = train_trans_eql.merge(  train_id_eql, on = 'TransactionID', how = 'outer')
test_clean  =  test_trans_eql.merge(  test_id_eql , on = 'TransactionID', how = 'outer')

print(train_clean.shape)

(20000, 401) (20000, 230)
(35116, 630)


In [14]:
# Fill with mean if missing
def DestroyMissing(df):
  df.fillna(df.mean(), inplace = True)
  df.fillna(df.mean(), inplace = True)


print(train_clean.shape)
train_clean.dropna(subset=['isFraud'], inplace=True)
DestroyMissing(train_clean)
print(train_clean.shape)

DestroyMissing(test_clean)
print(test_clean.shape)

(35116, 630)
(20000, 630)
(37027, 629)


## Saving clean files

In [15]:
train_clean.to_csv('/content/clean_train.csv', index = False)
print('/content/clean_train.csv')
test_clean.to_csv('/content/clean_test.csv', index = False)
print('/content/clean_test.csv')

/content/clean_train.csv
/content/clean_test.csv


# Dimension Reduction by Principal Component Analysis

## Training Data and Validation

In [0]:
# Standardise the X variables
def Standardise(X):
  X_Std = StandardScaler().fit_transform(X)
  X_Std = pd.DataFrame(X_Std, columns = X.columns)
  return X_Std

# Output PCA dataset
def PCA_DF(n_components, DF):
  pca = PCA(n_components)
  PrincComp = pca.fit_transform(DF)
  return pd.DataFrame(data = PrincComp)


# Evaluate recall of different PCAs to determine the best accuracy/computation tradeoff
def TestPCA(X_Std, y, a,b,c):
    LR = LogisticRegression()
    Eval = {'PCs':[], 'Recall':[]}
    X_Train, X_Test, Y_Train, Y_Test = train_test_split(X_Std, y, test_size=0.3)
    
    for PCs in range(a,b,c):
        print(PCs)
        
        # Create PCA dataframe
        Train_principalDf = PCA_DF(PCs, X_Train)
        Test_principalDf = PCA_DF(PCs, X_Test)
        
        # Use to train Logistic Regression model
        LR.fit(Train_principalDf, Y_Train)
        predictions = LR.predict(Test_principalDf)

        #Evaluate the model
        recall = recall_score(predictions, Y_Test)
        Eval['PCs'].append(PCs)
        Eval['Recall'].append(recall)

    # Return the recall scores for each number of PCs
    return pd.DataFrame(Eval['Recall'], index = Eval['PCs'])

In [17]:
train = pd.read_csv('/content/clean_train.csv')

# Separate predictors, target, and ID
ID = train[['TransactionID']]
X = train.drop('TransactionID', axis = 1)
y = train[['isFraud']]

# Output standardised dataset
X_Std = Standardise(X)
X_Std.head()

,V115,V288,P_emaildomain: hotmail.es,V59,P_emaildomain: juno.com,V119,TransactionAmt,C13,V21,P_emaildomain: att.net,V57,P_emaildomain: mail.com,M6: nan,V18,C5,P_emaildomain: roadrunner.com,P_emaildomain: earthlink.net,P_emaildomain: hotmail.fr,card5,V68,card4: discover,V58,V109,V34,card2,V290,V317,V305,V294,V73,V291,card4: nan,V319,V17,V71,V100,addr2,V309,V14,V95,V287,V282,card4: visa,V302,V121,V16,P_emaildomain: gmail.com,V67,V22,P_emaildomain: web.de,P_emaildomain: cableone.net,V108,P_emaildomain: live.fr,C4,V314,C1,ProductCD: C,V130,V126,V111,C8,V70,V24,V102,V15,ProductCD: H,V31,V62,P_emaildomain: bellsouth.net,P_emaildomain: ptd.net,V65,V110,V134,C12,V122,V298,V125,V26,V303,V315,V120,V28,V105,V293,ProductCD: W,card4: american express,P_emaildomain: outlook.es,card6: nan,V313,V20,P_emaildomain: windstream.net,V301,V124,V308,V103,V55,P_emaildomain: aol.com,P_emaildomain: frontiernet.net,P_emaildomain: yahoo.fr,card6: debit,P_emaildomain: centurylink.net,V61,V136,V66,P_emaildomain: charter.net,TransactionDT,V318,V320,V32,V306,P_emaildomain: msn.com,V64,P_emaildomain: aim.com,V53,P_emaildomain: twc.com,V112,ProductCD: S,P_emaildomain: hotmail.co.uk,P_emaildomain: suddenlink.net,V56,V304,ProductCD: nan,V131,V99,V127,P_emaildomain: live.com.mx,V311,addr1,P_emaildomain: embarqmail.com,P_emaildomain: outlook.com,V292,V137,P_emaildomain: hotmail.com,V286,P_emaildomain: cfl.rr.com,P_emaildomain: icloud.com,P_emaildomain: q.com,V135,C3,V98,card3,V101,V129,P_emaildomain: hotmail.de,P_emaildomain: optonline.net,M6: F,C11,P_emaildomain: anonymous.com,V96,V74,P_emaildomain: rocketmail.com,D1,V279,P_emaildomain: yahoo.co.uk,card4: mastercard,V312,C10,V299,V23,P_emaildomain: nan,V33,V27,V123,P_emaildomain: gmail,P_emaildomain: yahoo.com,V107,V114,P_emaildomain: frontier.com,V19,P_emaildomain: mac.com,V69,V13,V29,V316,P_emaildomain: yahoo.de,V281,P_emaildomain: netzero.com,P_emaildomain: live.com,V133,V310,V285,P_emaildomain: yahoo.es,C14,V307,V297,V106,V132,P_emaildomain: me.com,V30,V116,V25,V60,C6,P_emaildomain: yahoo.com.mx,P_emaildomain: cox.net,P_emaildomain: sbcglobal.net,V104,V128,V63,card1,V289,C7,V284,C9,V283,C2,P_emaildomain: ymail.com,V118,V300,P_emaildomain: comcast.net,P_emaildomain: verizon.net,V295,V72,V113,P_emaildomain: prodigy.net.mx,P_emaildomain: netzero.net,V296,V321,V12,P_emaildomain: sc.rr.com,V97,V117,V54,D10,ProductCD: R,card6: credit,M6: T,V280,isFraud,DeviceInfo: SM-J730G Build/NRD90M,id_31: ie,DeviceInfo: Moto Z2 Play Build/NPSS26.118-19-6,DeviceInfo: SAMSUNG SM-J320F Build/LMY47V,DeviceInfo: LG-D693n Build/KOT49I.V10a,DeviceInfo: Z981 Build/MMB29M,DeviceInfo: SM-E500M Build/KTU84P,DeviceInfo: SM-N920V Build/NRD90M,id_31: chrome 58.0,id_01,DeviceInfo: M4 SS4453 Build/MMB29M,id_31: chrome 56.0 for android,DeviceInfo: Moto E (4) Build/NMA26.42-69,id_31: chrome 62.0 for android,id_28: Found,DeviceInfo: SAMSUNG SM-J727T1 Build/NRD90M,DeviceInfo: E5606 Build/30.2.A.1.21,DeviceInfo: SM-G935V Build/NRD90M,DeviceInfo: SM-G925T Build/NRD90M,DeviceInfo: SM-G930T Build/NRD90M,DeviceInfo: SM-N900W8 Build/LRX21V,DeviceInfo: SAMSUNG SM-G920P Build/NRD90M,DeviceInfo: SM-A510M Build/NRD90M,DeviceInfo: LG-K220 Build/MXB48T,DeviceInfo: Moto G Play Build/MPIS24.241-15.3-7,DeviceInfo: SAMSUNG SM-G532M Build/MMB29T,DeviceInfo: Ilium L910 Build/MRA58K,DeviceInfo: Android 4.4.2,DeviceInfo: SM-G950W,DeviceInfo: WAS-LX3 Build/HUAWEIWAS-LX3,DeviceInfo: 5025G Build/LMY47I,DeviceInfo: XT1635-01,DeviceInfo: Moto Z2 Play Build/NPS26.118-19,DeviceInfo: BAC-L03 Build/HUAWEIBAC-L03,DeviceInfo: SM-J700M Build/MMB29K,DeviceInfo: KFAUWI Build/LVY48F,DeviceInfo: SAMSUNG SM-G890A Build/NRD90M,DeviceInfo: LG-TP450 Build/NRD90U,DeviceInfo: M4 SS4451 Build/LMY47D,id_12: Found,DeviceInfo: SAMSUNG SM-G530H Build/LRX22G,id_31: edge,id_28: nan,id_31: chrome 62.0,DeviceInfo: SM-T350 Build/MMB29M,id_31: chrome,DeviceInfo: SM-J120H Build/LMY47V,id_29: nan,DeviceInfo: SAMSUNG SM-A520F Build/NRD90M,DeviceInfo: SM-G900P Build/MMB29M,DeviceInfo: SM-A300H Build/LRX

In [0]:
#scores = TestPCA(X_Std, y, 90,200, 20)
#plt.plot(scores)

# Best PCA value is around 125

In [19]:
# Determine 125 PCA Variables is a good balance between accuracy and computation time
train_PCA = PCA_DF(125, X_Std)
train_PCA['TransactionID'] = ID
train_PCA['isFraud'] = y

train_PCA.to_csv('PCA_train.csv',  index = False)

print('Training Data has been cleaned and saved')

Training Data has been cleaned and saved


## Test Data

In [20]:
test = pd.read_csv('/content/clean_test.csv')

# Process Testing data, same as training
X_Std_Test = Standardise(test.drop('TransactionID', axis =1 ))

# Confirm the shapes are as they should be
a = {'StdTest ': X_Std_Test,
     'OrgTest ': test,
     'StdTrain': X_Std,
     'OrgTrain':X}

for i in a.keys():
    print(i, a[i].shape)

X_Std_Test.head()

StdTest  (37027, 628)
OrgTest  (37027, 629)
StdTrain (20000, 629)
OrgTrain (20000, 629)


,V115,V288,P_emaildomain: hotmail.es,V59,P_emaildomain: juno.com,V119,TransactionAmt,C13,V21,P_emaildomain: att.net,V57,P_emaildomain: mail.com,M6: nan,V18,C5,P_emaildomain: roadrunner.com,P_emaildomain: earthlink.net,P_emaildomain: hotmail.fr,card5,V68,card4: discover,V58,V109,V34,card2,V290,V317,V305,V294,V73,V291,card4: nan,V319,V17,V71,V100,addr2,V309,V14,V95,V287,V282,card4: visa,V302,V121,V16,P_emaildomain: gmail.com,V67,V22,P_emaildomain: web.de,P_emaildomain: cableone.net,V108,P_emaildomain: live.fr,C4,V314,C1,ProductCD: C,V130,V126,V111,C8,V70,V24,V102,V15,ProductCD: H,V31,V62,P_emaildomain: bellsouth.net,P_emaildomain: ptd.net,V65,V110,V134,C12,V122,V298,V125,V26,V303,V315,V120,V28,V105,V293,ProductCD: W,card4: american express,P_emaildomain: outlook.es,card6: nan,V313,V20,P_emaildomain: windstream.net,V301,V124,V308,V103,V55,P_emaildomain: aol.com,P_emaildomain: frontiernet.net,P_emaildomain: yahoo.fr,card6: debit,P_emaildomain: centurylink.net,V61,V136,V66,P_emaildomain: charter.net,TransactionDT,V318,V320,V32,V306,P_emaildomain: msn.com,V64,P_emaildomain: aim.com,V53,P_emaildomain: twc.com,V112,ProductCD: S,P_emaildomain: hotmail.co.uk,P_emaildomain: suddenlink.net,V56,V304,ProductCD: nan,V131,V99,V127,P_emaildomain: live.com.mx,V311,addr1,P_emaildomain: embarqmail.com,P_emaildomain: outlook.com,V292,V137,P_emaildomain: hotmail.com,V286,P_emaildomain: cfl.rr.com,P_emaildomain: icloud.com,P_emaildomain: q.com,V135,C3,V98,card3,V101,V129,P_emaildomain: hotmail.de,P_emaildomain: optonline.net,M6: F,C11,P_emaildomain: anonymous.com,V96,V74,P_emaildomain: rocketmail.com,D1,V279,P_emaildomain: yahoo.co.uk,card4: mastercard,V312,C10,V299,V23,P_emaildomain: nan,V33,V27,V123,P_emaildomain: gmail,P_emaildomain: yahoo.com,V107,V114,P_emaildomain: frontier.com,V19,P_emaildomain: mac.com,V69,V13,V29,V316,P_emaildomain: yahoo.de,V281,P_emaildomain: netzero.com,P_emaildomain: live.com,V133,V310,V285,P_emaildomain: yahoo.es,C14,V307,V297,V106,V132,P_emaildomain: me.com,V30,V116,V25,V60,C6,P_emaildomain: yahoo.com.mx,P_emaildomain: cox.net,P_emaildomain: sbcglobal.net,V104,V128,V63,card1,V289,C7,V284,C9,V283,C2,P_emaildomain: ymail.com,V118,V300,P_emaildomain: comcast.net,P_emaildomain: verizon.net,V295,V72,V113,P_emaildomain: prodigy.net.mx,P_emaildomain: netzero.net,V296,V321,V12,P_emaildomain: sc.rr.com,V97,V117,V54,D10,ProductCD: R,card6: credit,M6: T,V280,DeviceInfo: SM-J730G Build/NRD90M,id_31: ie,DeviceInfo: Moto Z2 Play Build/NPSS26.118-19-6,DeviceInfo: SAMSUNG SM-J320F Build/LMY47V,DeviceInfo: LG-D693n Build/KOT49I.V10a,DeviceInfo: Z981 Build/MMB29M,DeviceInfo: SM-E500M Build/KTU84P,DeviceInfo: SM-N920V Build/NRD90M,id_31: chrome 58.0,id_01,DeviceInfo: M4 SS4453 Build/MMB29M,id_31: chrome 56.0 for android,DeviceInfo: Moto E (4) Build/NMA26.42-69,id_31: chrome 62.0 for android,id_28: Found,DeviceInfo: SAMSUNG SM-J727T1 Build/NRD90M,DeviceInfo: E5606 Build/30.2.A.1.21,DeviceInfo: SM-G935V Build/NRD90M,DeviceInfo: SM-G925T Build/NRD90M,DeviceInfo: SM-G930T Build/NRD90M,DeviceInfo: SM-N900W8 Build/LRX21V,DeviceInfo: SAMSUNG SM-G920P Build/NRD90M,DeviceInfo: SM-A510M Build/NRD90M,DeviceInfo: LG-K220 Build/MXB48T,DeviceInfo: Moto G Play Build/MPIS24.241-15.3-7,DeviceInfo: SAMSUNG SM-G532M Build/MMB29T,DeviceInfo: Ilium L910 Build/MRA58K,DeviceInfo: Android 4.4.2,DeviceInfo: SM-G950W,DeviceInfo: WAS-LX3 Build/HUAWEIWAS-LX3,DeviceInfo: 5025G Build/LMY47I,DeviceInfo: XT1635-01,DeviceInfo: Moto Z2 Play Build/NPS26.118-19,DeviceInfo: BAC-L03 Build/HUAWEIBAC-L03,DeviceInfo: SM-J700M Build/MMB29K,DeviceInfo: KFAUWI Build/LVY48F,DeviceInfo: SAMSUNG SM-G890A Build/NRD90M,DeviceInfo: LG-TP450 Build/NRD90U,DeviceInfo: M4 SS4451 Build/LMY47D,id_12: Found,DeviceInfo: SAMSUNG SM-G530H Build/LRX22G,id_31: edge,id_28: nan,id_31: chrome 62.0,DeviceInfo: SM-T350 Build/MMB29M,id_31: chrome,DeviceInfo: SM-J120H Build/LMY47V,id_29: nan,DeviceInfo: SAMSUNG SM-A520F Build/NRD90M,DeviceInfo: SM-G900P Build/MMB29M,DeviceInfo: SM-A300H Build/LRX22G,id_1

In [21]:
test_PCA = PCA_DF(125, X_Std_Test)
test_PCA['TransactionID'] = test['TransactionID']
test_PCA.to_csv('TEST.csv',  index = False)

print('Testing Data has been cleaned and saved')

Testing Data has been cleaned and saved


# SMOTE

In [22]:
# train = pd.read_csv('/content/PCA_train.csv')

# Separate predictors, target, and ID
ID = train[['TransactionID']]
X = train.drop('TransactionID', axis = 1)
y = train[['isFraud']]

# Show the counts of each outcome
[print('Class {} has {} instances'.format(label, count))
 for label, count in zip(*np.unique(y, return_counts=True))]

Class 0.0 has 19438 instances
Class 1.0 has 562 instances


[None, None]

In [23]:
# Fit SMOTE
kmeans_smote = KMeansSMOTE(kmeans_args={'n_clusters': 100},
                           smote_args={'k_neighbors': 10})
X_resampled, y_resampled = kmeans_smote.fit_sample(X, y)

[print('Class {} has {} instances after oversampling'.format(label, count))
 for label, count in zip(*np.unique(y_resampled, return_counts=True))]

Class 0.0 has 19438 instances after oversampling
Class 1.0 has 19438 instances after oversampling


[None, None]

In [24]:
X_resampled, y_resampled = pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)

X_resampled.columns = X.columns
y_resampled.columns = ['y']

train_resampled = pd.concat([X_resampled,y_resampled, ID], axis = 1)
print(train_resampled.shape)
train_resampled.head()

(38876, 631)


,V115,V288,P_emaildomain: hotmail.es,V59,P_emaildomain: juno.com,V119,TransactionAmt,C13,V21,P_emaildomain: att.net,V57,P_emaildomain: mail.com,M6: nan,V18,C5,P_emaildomain: roadrunner.com,P_emaildomain: earthlink.net,P_emaildomain: hotmail.fr,card5,V68,card4: discover,V58,V109,V34,card2,V290,V317,V305,V294,V73,V291,card4: nan,V319,V17,V71,V100,addr2,V309,V14,V95,V287,V282,card4: visa,V302,V121,V16,P_emaildomain: gmail.com,V67,V22,P_emaildomain: web.de,P_emaildomain: cableone.net,V108,P_emaildomain: live.fr,C4,V314,C1,ProductCD: C,V130,V126,V111,C8,V70,V24,V102,V15,ProductCD: H,V31,V62,P_emaildomain: bellsouth.net,P_emaildomain: ptd.net,V65,V110,V134,C12,V122,V298,V125,V26,V303,V315,V120,V28,V105,V293,ProductCD: W,card4: american express,P_emaildomain: outlook.es,card6: nan,V313,V20,P_emaildomain: windstream.net,V301,V124,V308,V103,V55,P_emaildomain: aol.com,P_emaildomain: frontiernet.net,P_emaildomain: yahoo.fr,card6: debit,P_emaildomain: centurylink.net,V61,V136,V66,P_emaildomain: charter.net,TransactionDT,V318,V320,V32,V306,P_emaildomain: msn.com,V64,P_emaildomain: aim.com,V53,P_emaildomain: twc.com,V112,ProductCD: S,P_emaildomain: hotmail.co.uk,P_emaildomain: suddenlink.net,V56,V304,ProductCD: nan,V131,V99,V127,P_emaildomain: live.com.mx,V311,addr1,P_emaildomain: embarqmail.com,P_emaildomain: outlook.com,V292,V137,P_emaildomain: hotmail.com,V286,P_emaildomain: cfl.rr.com,P_emaildomain: icloud.com,P_emaildomain: q.com,V135,C3,V98,card3,V101,V129,P_emaildomain: hotmail.de,P_emaildomain: optonline.net,M6: F,C11,P_emaildomain: anonymous.com,V96,V74,P_emaildomain: rocketmail.com,D1,V279,P_emaildomain: yahoo.co.uk,card4: mastercard,V312,C10,V299,V23,P_emaildomain: nan,V33,V27,V123,P_emaildomain: gmail,P_emaildomain: yahoo.com,V107,V114,P_emaildomain: frontier.com,V19,P_emaildomain: mac.com,V69,V13,V29,V316,P_emaildomain: yahoo.de,V281,P_emaildomain: netzero.com,P_emaildomain: live.com,V133,V310,V285,P_emaildomain: yahoo.es,C14,V307,V297,V106,V132,P_emaildomain: me.com,V30,V116,V25,V60,C6,P_emaildomain: yahoo.com.mx,P_emaildomain: cox.net,P_emaildomain: sbcglobal.net,V104,V128,V63,card1,V289,C7,V284,C9,V283,C2,P_emaildomain: ymail.com,V118,V300,P_emaildomain: comcast.net,P_emaildomain: verizon.net,V295,V72,V113,P_emaildomain: prodigy.net.mx,P_emaildomain: netzero.net,V296,V321,V12,P_emaildomain: sc.rr.com,V97,V117,V54,D10,ProductCD: R,card6: credit,M6: T,V280,isFraud,DeviceInfo: SM-J730G Build/NRD90M,id_31: ie,DeviceInfo: Moto Z2 Play Build/NPSS26.118-19-6,DeviceInfo: SAMSUNG SM-J320F Build/LMY47V,DeviceInfo: LG-D693n Build/KOT49I.V10a,DeviceInfo: Z981 Build/MMB29M,DeviceInfo: SM-E500M Build/KTU84P,DeviceInfo: SM-N920V Build/NRD90M,id_31: chrome 58.0,id_01,DeviceInfo: M4 SS4453 Build/MMB29M,id_31: chrome 56.0 for android,DeviceInfo: Moto E (4) Build/NMA26.42-69,id_31: chrome 62.0 for android,id_28: Found,DeviceInfo: SAMSUNG SM-J727T1 Build/NRD90M,DeviceInfo: E5606 Build/30.2.A.1.21,DeviceInfo: SM-G935V Build/NRD90M,DeviceInfo: SM-G925T Build/NRD90M,DeviceInfo: SM-G930T Build/NRD90M,DeviceInfo: SM-N900W8 Build/LRX21V,DeviceInfo: SAMSUNG SM-G920P Build/NRD90M,DeviceInfo: SM-A510M Build/NRD90M,DeviceInfo: LG-K220 Build/MXB48T,DeviceInfo: Moto G Play Build/MPIS24.241-15.3-7,DeviceInfo: SAMSUNG SM-G532M Build/MMB29T,DeviceInfo: Ilium L910 Build/MRA58K,DeviceInfo: Android 4.4.2,DeviceInfo: SM-G950W,DeviceInfo: WAS-LX3 Build/HUAWEIWAS-LX3,DeviceInfo: 5025G Build/LMY47I,DeviceInfo: XT1635-01,DeviceInfo: Moto Z2 Play Build/NPS26.118-19,DeviceInfo: BAC-L03 Build/HUAWEIBAC-L03,DeviceInfo: SM-J700M Build/MMB29K,DeviceInfo: KFAUWI Build/LVY48F,DeviceInfo: SAMSUNG SM-G890A Build/NRD90M,DeviceInfo: LG-TP450 Build/NRD90U,DeviceInfo: M4 SS4451 Build/LMY47D,id_12: Found,DeviceInfo: SAMSUNG SM-G530H Build/LRX22G,id_31: edge,id_28: nan,id_31: chrome 62.0,DeviceInfo: SM-T350 Build/MMB29M,id_31: chrome,DeviceInfo: SM-J120H Build/LMY47V,id_29: nan,DeviceInfo: SAMSUNG SM-A520F Build/NRD90M,DeviceInfo: SM-G900P Build/MMB29M,DeviceInfo: SM-A300H Build/LRX

## Save Resampled data

In [0]:
train_resampled.to_csv('TRAIN.csv',  index = False)

# Modelling

In [26]:
train = pd.read_csv('/content/TRAIN.csv')
print(train.shape)
test = pd.read_csv('/content/TEST.csv')
print(test.shape)

(38876, 631)
(37027, 126)


In [0]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
ID = train[['TransactionID']]
X = train.drop(['TransactionID', 'isFraud'], axis = 1)
y = train[['isFraud']]

clf = LogisticRegressionCV(cv=5, random_state=0).fit(X, y)

# Dictionary --> {isFraud: [Fold][Cs]}

In [46]:
# Test with training observations
xobs = pd.Series(X.loc[1,:])
clf.predict([xobs])

array([1.])

# Evaluation

In [52]:
# Evaluate model on training data
sub = ID
sub['True'] = y
sub['Pred'] = clf.predict(X)

# Accuracy
accuracy = np.where(sub['Pred'] == sub['True'], 1, 0)
print('Accuracy =', sum(accuracy)/ len(accuracy))

# Recall
TP = np.where((sub['Pred'] == sub['True']) & (sub['True'] == 1), 1, 0)
FN = np.where((sub['Pred'] != sub['True']) & (sub['True'] == 1), 1, 0)
print('Recall =', sum(TP)/ (sum(TP) + sum(FN)))

sub.head()

Accuracy = 0.7218077991562918
Recall = 0.7039818911410639


,TransactionID,Pred,True
0,2987000.0,0.0,0.0
1,2987001.0,1.0,0.0
2,2987002.0,1.0,0.0
3,2987003.0,0.0,0.0
4,2987004.0,0.0,0.0
